# MoSDeF - A Molecular Simulation Design Framework

## Foyer Overview
Foyer is designed to be a force field agnostic tool for performing atom-typing that can output to various file formats. That is, the goal is to create a single, flexible tool that can work with a range of force fields and simulation engines, rather than creating software specific to a single forcefield/simulation engine. 

A key goal of Foyer is to allow force field usage to be unambiguous and the abilication of force fields to be reproducible.  Foyer employs a force field file format that is:
- simultaneously human and machine readable
- can be automatically validated
- easy to disseminate 
- easy to evolve and expand.

### File Format
Foyer force fields are defined within an XML file that contains both the 'rules' required for atomtyping as well as the force field parameters within a single file. 

The Foyer XML format is an extension of the [OpenMM forcefield XML format](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields). The only differences reside in the `AtomTypes` section, where several additional attributes are available (which we will examine in a moment) that allow for atomtyping.

The `AtomTypes` section of the Foyer XML is similar to that used for OpenMM forcefield XMLs; however, each `Type` in Foyer XML supports four additional attributes not found in OpenMM:
* `def` - SMARTS string describing the chemical substructure of this atom type, as discussed later (Follow [this link](https://github.com/mosdef-hub/foyer/blob/master/docs/smarts.md) for more on SMARTS-based atomtyping using Foyer.)
* `desc` - Brief description of the atomtype
* `doi` - DOI reference for parameters associated with this atomtype
* `overrides` - One or more atomtypes to 'override', providing precedence to this atomtype (discussed later)


Since all the usage rules are encoded in a text file outside of the software itself, rules can be evolved and corrected without changing the software itself; it also makes it easier to develop and distribute new force fields and subsets specific to a project/manuscript. 


Let's quickly look at a Foyer XML file:

In [1]:
!cat utils/oplsaa-alcohol.xml

<ForceField>
  <AtomTypes>
    <Type name="opls_135" class="CT" element="C" mass="12.01100"
          def="[C;X4](C)(H)(H)H" desc="alkane CH3" doi="10.1063/1.474052"/>
    <Type name="opls_136" class="CT" element="C" mass="12.01100"
          def="[C;X4](C)(C)(H)H" desc="alkane CH2" doi="10.1063/1.474052"/>
    <Type name="opls_140" class="HC" element="H" mass="1.00800"
          def="H[C;X4]" desc="alkane H" doi="10.1021/ja9621760"/>
    <Type name="opls_154" class="OH" element="O" mass="15.9994"
          def="[O;X2](C)H" desc="all-atom O: mono alcohols"
          doi="10.1021/ja9621760"/>
    <Type name="opls_155" class="HO" element="H" mass="1.00800"
          def="H[O;%opls_154]" desc="all-atom H(O): mono alcohols"
          doi="10.1021/ja9621760"/>
    <Type name="opls_157" class="CT" element="C" mass="12.01100"
          def="[C;X4](H)(H)(C)[O;%opls_154]"
          desc="all-atom C: CH3 and CH2, alcohols"
          overrides="opls_136" doi="10.1021/ja9621760"/>
  </AtomTypes>
 

## Defining Chemical Context (i.e., "Rules") using SMARTS

Classical force fields are typically able to achieve high accuracy by creating sets of highly specific fitting parameters (i.e., atom types), in which each atom type describes an interaction site within a different chemical context. The chemical context is typically defined by the bonded environment of an interaction site (e.g., the number of bonds and the identity of the bonded neighbors) and may also consider, among other factors, the bonded environment of the neighbors, and/or the specific molecule/structure within which the interaction site is included.  Chemical context effectively defines the 'rules' for when an atomtype should apply. 

Foyer relies upon using [SMARTS](http://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) to define the chemical context of an atom-type. SMARTS is a language for describing chemical structures and substructures.  In Foyer we tend focus on substructures, rather than trying to describe the entire molecule in a single string, given that the goal is to uniquely describe differentiate each interaction site/atom-type in the system.



### Example, Methane: 
Consider defining the chemical context of the atoms (i.e., interactions sites) in a methane molecule ($CH_4$) using SMARTS.  The first thing to note is that there are multiple valid SMARTS that could describe the atom.  

#### Carbon:
- For example, the simplest way to describe the carbon atom in methane would be:
 - ```C``` 
- While in some cases such a simple definition may be sufficient, typically we wish to also define the number of bond, e.g.:
 - ```[C;X4]```
 - Note, ```;``` represents ```AND```, thus stating that the atom is a carbon AND it has 4 bonds. 
- Often it is useful to further define the identity of those bonds. In this case, we would typically describe methane as: 
 - ```[C;X4](H)(H)(H)(H)``` 

#### Hydrogen:
- Since hydrogen can only form a single bond, we could simply define it as:
 - ```H```
 - however, this may be insufficient as the parameters will likely be different based on what it is bonded to. 
- Additional specificity can be encoded by providing information about the neighbors, e.g.:
 - ```H(C)``` 
 - this specifies this hydrogen must be bonded to a carbon.
- This could be made further specific by further specifyin that this is a hydrogen that is bonded to a carbon with four bonds. 
 -  ```H[C;X4]```

### Exercise 1:

#### Setting up a force field
Modify the "mff_test1.xml" file (in the "mff_test1" directory) defining SMARTS strings that appropriately describe the atom types for simple linear alkanes based on our fictious forcefield "mosdef_ff". 

Here, the atom types for "mosdef_ff" are:

* `mff_0`: carbon in methane (CH_4)
* `mff_1`: carbon in a terminal methyl group of a linear chain (-CH_3)
* `mff_2`: carbon in a middle methyl group of a linear chain (-CH_2-)
* `mff_3`: a generic hydrogen atom

<img src="./utils/mff.png" alt="Drawing" style="width: 700px;"/>


After defining the appropriate SMARTS strings, run the test suite (below) to ensure SMARTS are correctly encoded.  This uses pytest to compare the output of your forcefield to the atom types listed in known mol2 files for methane, ethane, and propane. 

Note, don't worry about the bonds/angles/dihedrals listed in the file.  Those are just to avoid warnings about an underspecified force field (Foyer will provide warnings if connected pairs/triples/quartets are identified, but not parameterized as bond/angles/dihedrals).

In [3]:
%cd mff_test1/
!py.test -v --tb=line
%cd ..

/Users/cri/Projects/mosdef_tutorials/mff_test1
============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-4.3.0, py-1.7.0, pluggy-0.8.0 -- /Users/cri/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/cri/Projects/mosdef_tutorials/mff_test1, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.1.3, arraydiff-0.3
collected 3 items                                                              

test_atomtyping.py::test_atomtyping[test_molecules/ethane.mol2] PASSED   [ 33%]
test_atomtyping.py::test_atomtyping[test_molecules/propane.mol2] PASSED  [ 66%]
test_atomtyping.py::test_atomtyping[test_molecules/methane.mol2] PASSED  [100%]

=============================== warnings summary ===============================
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758
  /Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\.'
 

### Working with coarse-grained and united atom forcefields

Foyer allows non-atomistic types to be defined within SMARTS, allowing coarse-grained and united atom forcefields to be handled as well. Non-elemental species can easily be defined by pre-pending the name of custom "element" with an underscore.

For example, let us consider defining the SMARTS for a simple homopolymer composed of "A" beads, where terminal groups are of atom type "cg_term" and middle groups are "cg_mid". Here, the key differentiation between atom types is the number of bonds (1 for cg_term, 2 for cg_mid): 

- `cg_term` : `[_A;X1]`
- `cg_mid` : `[_A;X2]`

Similarly, the following lines could be used to describe beads representing $-CH_2-$ groups in a polymer using the TraPPE forcefield. 

`  <Type name="CH2_sp3" class="CH2" element="_CH2" mass="14.02700" 
   def="[_CH2;X2]([_CH3,_CH2])[_CH3,_CH2]" 
   desc="Alkane CH2, united atom" doi="10.1021/jp972543+"/>`
  
Here, the SMARTS definition `[_CH2;X2]([_CH3,_CH2])[_CH3,_CH2]` states that for atom-type `CH2_sp3`

- our bead is `_CH2` with 2 bonded neighbors, i.e., `[_CH2;X2]`
- those neighbors can be either `_CH2` or `_CH3`, since, i.e., `([_CH3,_CH2])[_CH3,_CH2]` (note, `,`  represents an "or" statement).



### Using  `overrides` to set rule precendence

Force fields often contain atom types that ostensively have matching chemical context (at least in term of local bonded environment), but require different parameters.  For example, consider defining the a force field for both alkenes and benzene in a single file for the OPLS force field (note, for clarity only only 2 atom types are included):

`
<ForceField>
  <AtomTypes>
    <Type name="opls_142" class="CM" element="C" mass="12.01100"\\
        def="[C;X3](C)(C)H" desc="alkene C (RH-C=)"/>
    <Type name="opls_145" class="CA" element="C" mass="12.01100"\\
        def="[C;X3;r6]1[C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6]1"\\
        overrides="opls_142"/>
  </AtomTypes>
</ForceField>
`

When atom-typing a benzene molecule, the carbon atoms in the ring will match the SMARTS patterns for both `opls_142` (an alkene carbon) and `opls_145` (a benzene carbon). 


`Foyer` allows rule precedence to be explicitly stated via the use of the `overrides` attribute added to the XML file format. This allows atom type usage rules to be encoded in any order within the file, eliminating incorrectly placed rule order as a source of error. This approach also means that all rules are evaluated, helping to better ensure completeness and remove ambiguity.  Providing the `overrides` indicates that if the `opls_145` pattern matches, it will supersede `opls_142`.


`Foyer` iteratively evaluates all rules on all interaction sites in the system, maintaining for each interaction site a "whitelist" consisting of rules that evaluate to `True` and a "blacklist" consisting  of rules that have been superseded by another rule (i.e., those that appear in the `overrides` attribute). The set difference between the white- and blacklists of an interaction site yields the correct atom type if the force field is implemented correctly. Thus, in this example the difference between the whitelist (containing `opls_142` and `opls_145`) and blacklist (containing only `opls_142`) would be `opls_145`.


## Exercise 2
Let us modify the prior fictious "mosdef_ff" to include rules that will require overrides. 

Here we will add a new atom type `mff_4` which is a generic carbon that should be used when more specific matches are not found. 

Here, the atom types for "mosdef_ff" are:

* `mff_0`: carbon in methane (CH_4)
* `mff_1`: carbon in a terminal methyl group of a linear chain (-CH_3)
* `mff_2`: carbon in a middle methyl group of a linear chain (-CH_2-)
* `mff_3`: a hydrogen atom 
* `mff_4`: a generic carbon

Here, we have an additional test case that is simply a linear chain of connected carbons ("linear.mol2"). 

Make these edits to "mff_test2.xml" (in the "mff_test2" directory).  Here is an example of encoding an override.

```
<Type name="opls_138" def="[C;X4](H)(H)(H)H"
    class="CT" element="C" mass="12.01100" desc="alkane CH4"
    doi="10.1021/ja9621760" overrides="opls_135,opls_136"/>
```

When complete, run pytest below, as before.  
#### Try to break the force field
Assuming you did everything correct, modify the override statement of mff_4 such that you will cause the methane test to fail and examine the "FAILURES" reported by Foyer. 

In [9]:
%cd mff_test2/
!py.test -v --tb=line
%cd ..

/Users/cri/Projects/mosdef_tutorials/mff_test2
============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-4.3.0, py-1.7.0, pluggy-0.8.0 -- /Users/cri/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/cri/Projects/mosdef_tutorials/mff_test2, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.1.3, arraydiff-0.3
collected 4 items                                                              

test_atomtyping.py::test_atomtyping[test_molecules/ethane.mol2] PASSED   [ 25%]
test_atomtyping.py::test_atomtyping[test_molecules/linear.mol2] PASSED   [ 50%]
test_atomtyping.py::test_atomtyping[test_molecules/propane.mol2] PASSED  [ 75%]
test_atomtyping.py::test_atomtyping[test_molecules/methane.mol2] FAILED  [100%]

=================================== FAILURES ===================================
/Users/cri/anaconda3/lib/python3.6/site-packages/foyer/atomtyper.py:101: foyer.exceptions.FoyerError: Found multiple 

## Force field application with `Foyer`. Generating data files.

If we wanted to actually run a simulation of any of these systems we've built with mBuild, we would need to apply a force field and write the necessary data files. mBuild handles all of this through a single `save` command, where we can pass as arguments the name of the force field to apply (which uses `Foyer` under the hood) and the name of the file to create, which will be formatted based on the extension.


First, let's consider how we would write to Gromacs `TOP` and `GRO` formats.

The `GRO` format contains no force field information, so we do not have to pass a force field file to `save` when writing to this format. Let's start by using the "mosdef_ff" force field we set up previously to parameterize octane. 

In [ ]:
import mbuild as mb
from mbuild.lib.atoms import H
from mbuild.lib.moieties import CH2


class Alkane(mb.Compound):
    def __init__(self, chain_length=1):
        super(Alkane, self).__init__()
        hydrogen = H()
        last_monomer = CH2()
        # top capping CH2 -> CH3
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['up'])
        # Add to our `self` compound
        self.add(hydrogen)
        self.add(last_monomer)
        # loop over n = chain_length CH2's to add to Alkane
        for _ in range(chain_length-1):
            current_monomer = CH2()
            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer=current_monomer
        # bottom cap
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['down'])
        self.add(hydrogen)

octane = Alkane(chain_length=8)


octane.save('octane_test.gro', overwrite=True)
octane.save('octane_test.top', forcefield_files='mff_test2/mff_test2.xml', overwrite=True, combining_rule='geometric')

We can easily inspect the output:

In [15]:
!cat octane_test.top

;
;   File octane_test.top  was generated
;   By user: cri (503)
;   On host: IacBook-Pro-Retina.local
;   At date: Fri. March  5 16:24:46 2019
;
;   This is a standalone topology file
;
;   Created by:
;   ParmEd:       ipykernel_launcher.py, VERSION 3.1.0
;   Executable:   ipykernel_launcher.py
;   Library dir:  /usr/local/gromacs/share/gromacs/top
;   Command line:
;     /Users/cri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /Users/cri/Library/Jupyter/runtime/kernel-f447ecc3-5e1a-41fb-a516-12b21f028cda.json
;

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               3               no              1            0.5         

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
mff_3          1   1.007947  0.00000000  A          0.312        0.71128
mff_1          6  12.010780  0.00000000  A           0.35       0.276144
mff_2          6  12.010780  0.00000000  A           0.25        0.12552


[ moleculetype

If we wish to setup an entire system, we can also specify a `residues` argument. In this case, we are saying to treat every `Compound` with the name `Octane` as a separate residue. This helps to speed up the calculation by avoiding duplicating the work, since the box is effectively filled with duplicates of the same molecule.  


With this force field XML file, Foyer will use the SMARTS strings to atom-type our system and will then apply the proper force field parameters. We'll execute the `save` method again, this time passing through our force field file and changing the desired file format from GRO to TOP. Additionally, as OPLS uses geometric mixing rules as opposed to Lorentz-Berthelot, we can feed this to `save` as well.

**Note:** The warning message about unparameterized impropers can be safely ignored, as OPLS does not include any impropers for our system. By default, Foyer will warn the user if improper parameters are not specified for all possible impropers and will exit with an error if bond, angle, or dihedral parameters are not specified for all possible bonds, angles, dihedrals. (This behavior may be overridden if desired.)

In [18]:
octane_box = mb.fill_box(octane, n_compounds=10, box=[3, 3, 3], seed=2)

octane_box.save('octane_system.gro', residues='Octane', overwrite=True)
octane_box.save('octane_system.top', forcefield_files='mff_test2/mff_test2.xml', residues='Octane',
                 combining_rule='geometric', overwrite=True)

/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\#'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\['
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\]'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\('
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\)'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationW

Finally, just to prove that these files were written correctly, we can take a quick peek.

In [13]:
!cat octane_system.top

;
;   File system.top  was generated
;   By user: cri (503)
;   On host: IacBook-Pro-Retina.local
;   At date: Fri. March  5 16:19:01 2019
;
;   This is a standalone topology file
;
;   Created by:
;   ParmEd:       ipykernel_launcher.py, VERSION 3.1.0
;   Executable:   ipykernel_launcher.py
;   Library dir:  /usr/local/gromacs/share/gromacs/top
;   Command line:
;     /Users/cri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /Users/cri/Library/Jupyter/runtime/kernel-f447ecc3-5e1a-41fb-a516-12b21f028cda.json
;

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               3               no              1            0.5         

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
opls_140        1   1.007947  0.00000000  A           0.25        0.12552
opls_135        6  12.010780  0.00000000  A           0.35       0.276144
opls_136        6  12.010780  0.00000000  A           0.35       0.276144


[ moleculetype ]

We can also trivially change the force field we apply.  For example, instead of our made up force field, let us apply the OPLS_AA forcefield.  We have included a file that includes parameters for alkanes and alcohols which would apply to this system.  We viewed this file previously.  The only change in the save command below (aside from changing the name of the top file) is the forcefield_files argument we pass to the function.


In [19]:
!cat utils/oplsaa-alcohol.xml

<ForceField>
  <AtomTypes>
    <Type name="opls_135" class="CT" element="C" mass="12.01100"
          def="[C;X4](C)(H)(H)H" desc="alkane CH3" doi="10.1063/1.474052"/>
    <Type name="opls_136" class="CT" element="C" mass="12.01100"
          def="[C;X4](C)(C)(H)H" desc="alkane CH2" doi="10.1063/1.474052"/>
    <Type name="opls_140" class="HC" element="H" mass="1.00800"
          def="H[C;X4]" desc="alkane H" doi="10.1021/ja9621760"/>
    <Type name="opls_154" class="OH" element="O" mass="15.9994"
          def="[O;X2](C)H" desc="all-atom O: mono alcohols"
          doi="10.1021/ja9621760"/>
    <Type name="opls_155" class="HO" element="H" mass="1.00800"
          def="H[O;%opls_154]" desc="all-atom H(O): mono alcohols"
          doi="10.1021/ja9621760"/>
    <Type name="opls_157" class="CT" element="C" mass="12.01100"
          def="[C;X4](H)(H)(C)[O;%opls_154]"
          desc="all-atom C: CH3 and CH2, alcohols"
          overrides="opls_136" doi="10.1021/ja9621760"/>
  </AtomTypes>
 

In [20]:
octane_box.save('octane_system_opls.top', forcefield_files='utils/oplsaa-alcohol.xml', residues='Octane',
                 combining_rule='geometric', overwrite=True)

/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\#'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\['
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\]'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\('
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\)'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/anaconda3/lib/python3.6/site-packages/plyplus/plyplus.py:758: DeprecationW

In [21]:
!cat octane_system_opls.top

;
;   File octane_system_opls.top  was generated
;   By user: cri (503)
;   On host: IacBook-Pro-Retina.local
;   At date: Fri. March  5 16:30:24 2019
;
;   This is a standalone topology file
;
;   Created by:
;   ParmEd:       ipykernel_launcher.py, VERSION 3.1.0
;   Executable:   ipykernel_launcher.py
;   Library dir:  /usr/local/gromacs/share/gromacs/top
;   Command line:
;     /Users/cri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /Users/cri/Library/Jupyter/runtime/kernel-f447ecc3-5e1a-41fb-a516-12b21f028cda.json
;

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               3               no              1            0.5         

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
opls_140        1   1.007947  0.00000000  A           0.25        0.12552
opls_135        6  12.010780  0.00000000  A           0.35       0.276144
opls_136        6  12.010780  0.00000000  A           0.35       0.276144


[ mo